Jesus D Serpa P

## Topología de la red.

1. Construir un clase  que permita definir una red neuronal con la topología
deseada y la función de activación para cada capa, para ello deberá construir una funcion Topology con el número de capas de la red neuronal :

Topology = [n_x, n_h1, n_h2, n_h3, ...,n_y]

En este caso:
- $n^{[0]}=n_x$ seran los valores de entradas de la capa de entrada
- $n^{[1]}=n_{h1}$ Primera capa oculta de la red neuronal
- $n^{[2]}=n_{h2}$ Segunda capa oculta de la red neuronal

.

.

.


- $n^{[l]}=n_{hl}$ Segunda capa oculta de la red neuronal
.

.

.

- $n^{[L]}=n_{y}$ Segunda capa oculta de la red neuronal

donde

- $\mathrm{n_x}$: valores de entrada
- $\mathrm{n_{h1}}$: hidden layer 1
- $\mathrm{n_{h2}}$: hidden layer 2
- $\mathrm{n_y}$: last layer

- $n^{[L]}=n_{y}$ Segunda capa oculta de la red neuronal


También definir una lista con las funciones de activaciones para cada capa.


activation=[None, relu, relu, relu, ...,sigmoid]

  


a. Cada unas de las capas deberá tener los parámetros de inicialización de manera aleatoria:


La matriz de parametros para cada capa debera tener:


$\mathrm{dim(\vec{b}^{[l]})}=n^{[l]}$

$\mathrm{dim(\vec{\Theta}^{[l]})}=n^{[l]}\times n^{[l-1]}$

Lo anteriores parametros deberán estar en el constructor de la clase.


b. Construya un metodo llamado output cuya salida serán los valores de Z y A


$\mathrm{dim(\vec{\cal{A}}^{[l]})}=n^{[l-1]}\times m $

$\mathrm{dim(\vec{\cal{Z}}^{[l]})}=n^{[l]}\times m $.

In [2]:
import numpy as np

class layer_nn:
    def __init__(self, act_fun, nlayer_present, nlayer_before):
        """
        act_fun: string con el nombre de la función de activación ("sigmoid", "tanh", "relu", ...)
        nlayer_present: n^{[l]}  (número de neuronas de esta capa)
        nlayer_before:  n^{[l-1]} (número de neuronas de la capa anterior)
        """
        # Theta^{[l]} ~ n^{[l]} x n^{[l-1]}
        self.theta = 2 * np.random.random((nlayer_present, nlayer_before)) - 1
        # b^{[l]} ~ n^{[l]} x 1
        self.b = 2 * np.random.random((nlayer_present, 1)) - 1

        self.act_fun = act_fun

        # Para almacenar resultados del forward:
        self.Z = None   # Z^{[l]}
        self.A = None   # A^{[l]}
        self.dA = None  # derivada de activación f'(Z^{[l]}) (la usaremos en backprop)

    def output(self, Z, A, dA):
        """
        Guarda en la capa los valores de Z^{[l]}, A^{[l]} y f'(Z^{[l]})
        """
        self.Z = Z
        self.A = A
        self.dA = dA


2. Construir un generalizacion de la red, en el que entrada el valor inicial
y la red neuronal completa arroje la salida y la actualizacion de la red con los parametros deseados:

  ```
  A, nn = forward_pass(A0, nn_red)

 ```

In [13]:
def act_function(Z, activation):
    if activation == "sigmoid":
        return 1 / (1 + np.exp(-Z))
    elif activation == "tanh":
        return np.tanh(Z)
    elif activation == "relu":
        return np.maximum(0, Z)
    else:
        # para la capa de entrada o identidad
        return Z

topology    = [12288, 3, 4, 6, 1]
activations = [None, "sigmoid", "sigmoid", "sigmoid", "sigmoid"]

nn_red = []
for l in range(1, len(topology)):
    layer = layer_nn(
        act_fun=activations[l],
        nlayer_present=topology[l],
        nlayer_before=topology[l-1]
    )
    nn_red.append(layer)

def forward_pass(A0, nn_red):
    """
    A0     : matriz de entrada (n^{[0]} x m)
    nn_red : lista de capas (objetos layer_nn)

    Salida:
      A  : salida de la última capa (n^{[L]} x m)
      nn : red actualizada con Z^{[l]} y A^{[l]} guardados en cada capa
    """
    A = A0
    updated_nn = []

    for layer in nn_red:
        # Z^{[l]} = Θ^{[l]} A^{[l-1]} + b^{[l]}
        Z = layer.theta @ A + layer.b     # (n^{[l]} x m)

        # A^{[l]} = f(Z^{[l]})
        A = act_function(Z, layer.act_fun)  # (n^{[l]} x m)

        # Guardar Z^{[l]} y A^{[l]} en la capa
        layer.output(Z, A, None)

        updated_nn.append(layer)

    return A, updated_nn

In [14]:
m  = 5                     # número de ejemplos (solo para prueba)
n_x = topology[0]          # tamaño de la entrada

# Entrada A0 con la forma correcta (n_x x m)
A0 = np.random.rand(n_x, m)

# Forward de toda la red
AL, nn = forward_pass(A0, nn_red)

print("Forma de A0 (entrada):", A0.shape)
print("Forma de salida AL   :", AL.shape)

Forma de A0 (entrada): (12288, 5)
Forma de salida AL   : (1, 5)


3. Encontrar la funcion de coste.


$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$



In [15]:
def cost_function(AL, Y):
    """
    AL : salida de la última capa, de dimensión (1, m)
    Y  : etiquetas verdaderas, de dimensión (1, m)

    Devuelve:
        cost : escalar con el valor de la función de coste
    """
    m = Y.shape[1]          # número de ejemplos

    # Evitar log(0)
    eps = 1e-15
    AL_clipped = np.clip(AL, eps, 1 - eps)

    cost = - (1.0 / m) * np.sum(
        Y * np.log(AL_clipped) + (1 - Y) * np.log(1 - AL_clipped)
    )

    return cost


4. Construir un codigo que permita realizar el BackwardPropagation


In [16]:
def act_derivative(Z, A, activation):
    """
    Z : Z^{[l]}  (n^{[l]} x m)
    A : A^{[l]}  (n^{[l]} x m)
    """
    if activation == "sigmoid":
        # f(z) = A  =>  f'(z) = A(1-A)
        return A * (1 - A)
    elif activation == "tanh":
        # f(z) = tanh(z)  =>  f'(z) = 1 - A^2
        return 1 - A**2
    elif activation == "relu":
        # f(z) = max(0,z)
        return (Z > 0).astype(float)
    else:
        # identidad
        return np.ones_like(Z)


In [17]:
def backward_pass(AL, Y, nn_red, A0):
    """
    AL    : salida de la última capa (n^{[L]} x m), normalmente (1 x m)
    Y     : etiquetas verdaderas (1 x m)
    nn_red: lista de capas (objetos layer_nn) de la red
    A0    : entrada de la red (n^{[0]} x m)

    Calcula y guarda en cada capa:
        layer.dtheta  ~ dΘ^{[l]}
        layer.db      ~ db^{[l]}

    Devuelve:
        nn_red actualizado con los gradientes.
    """
    m = Y.shape[1]

    # A^{[0]},...,A^{[L]}
    A_caches = [A0] + [layer.A for layer in nn_red]   # len = L+1
    L = len(nn_red)

    #  Gradiente en la salida
    # dJ/dAL (general, sin la simplificación A-Y)
    eps = 1e-15
    AL_clip = np.clip(AL, eps, 1 - eps)
    dAL = -(np.divide(Y, AL_clip) - np.divide(1 - Y, 1 - AL_clip))

    dA = dAL

    # Recorrido hacia atrás

    for l in reversed(range(L)):   # L-1, L-2, ..., 0
        layer = nn_red[l]

        Zl     = layer.Z          # Z^{[l+1]} en notación 0-index
        Al     = layer.A          # A^{[l+1]}
        A_prev = A_caches[l]      # A^{[l]}

        # Derivada de la activación
        g_prime = act_derivative(Zl, Al, layer.act_fun)

        # dZ^{[l+1]} = dA^{[l+1]} ⊙ g'(Z^{[l+1]})
        dZ = dA * g_prime                           # (n^{[l+1]} x m)

        # dΘ^{[l+1]} = (1/m) dZ^{[l+1]} A^{[l]T}
        dtheta = (1.0 / m) * (dZ @ A_prev.T)        # (n^{[l+1]} x n^{[l]})

        # db^{[l+1]} = (1/m) sum_j dZ^{[l+1]}_j
        db = (1.0 / m) * np.sum(dZ, axis=1, keepdims=True)  # (n^{[l+1]} x 1)

        # dA^{[l]} = Θ^{[l+1]T} dZ^{[l+1]}  (para la capa anterior)
        dA = layer.theta.T @ dZ                     # (n^{[l]} x m)

        # Guardar gradientes en la capa
        layer.dtheta = dtheta
        layer.db     = db

    return nn_red


In [21]:
AL, nn = forward_pass(A0, nn_red)   # AL: salida (1 x m)

# Define Y (true labels) with the correct shape (1, m)
m = AL.shape[1]  # Get m from the shape of AL
Y = np.random.randint(0, 2, size=(1, m)) # Example: binary labels for m samples

cost = cost_function(AL, Y)         # del punto 3
nn    = backward_pass(AL, Y, nn, A0)



In [22]:
def update_params(nn_red, alpha):
    """
    Actualiza los parámetros de la red usando los gradientes
    calculados en backward_pass.
    """
    for layer in nn_red:
        layer.theta = layer.theta - alpha * layer.dtheta
        layer.b     = layer.b     - alpha * layer.db


In [23]:
# Forward
AL, nn = forward_pass(A0, nn_red)

# Costo (punto 3)
J = cost_function(AL, Y)

# BackwardPropagation (punto 4)
nn = backward_pass(AL, Y, nn, A0)

# Actualización de parámetros (gradiente descendente)
alpha = 0.01
update_params(nn, alpha)
